In [6]:
from py010parser import parse_file, parse_string, c_ast

In [7]:
ast = parse_file("gif.bt", cpp_args="-xc++")  # cpp arg is needed on a Mac

In [8]:
import string

In [9]:
def char(c):
    if 32 <= ord(c) <= 96:
        return repr(c)
    return f"'\\0x{ord(c):02x}'"

In [15]:
class BT2FandangoVisitor(c_ast.NodeVisitor):
    def __init__(self):
        self.defs = ""
        self.forced_bytes = []

    # def visit(self, node):
    #     method_name = 'visit_' + node.__class__.__name__
    #     visitor = getattr(self, method_name, self.generic_visit)
    #     return visitor(node)

    def generic_visit(self, node) -> str:
        return " ".join(self.visit(child) for _, child in node.children())

    def visit_FileAST(self, node: c_ast.FileAST) -> str:
        members = ""
        for _, child in node.children():
            elem = self.visit(child)
            if elem:
                members += elem + "\n"
        start = f'\n<start> ::= {members};\n'
        return self.defs + start

    def visit_Typedef(self, node: c_ast.Typedef) -> str:
        members = ""
        n_members = 1
        for _, child in node.children():
            elem = self.visit(child)
            if elem and members:
                members += " "
                n_members += 1
            members += f"{elem}"
        if node.name:
            self.defs += f"<{node.name}> ::= {members};\n"
            if n_members > 1:
                self.defs += '\n'
        return ""

    def visit_Struct(self, node: c_ast.Struct) -> str:
        members = ""
        self.defs += '\n'
        for _, child in node.children():
            if self.forced_bytes:
                if members:
                    members += " "
                members += f"{char(self.forced_bytes[0])}"
                self.forced_bytes = self.forced_bytes[1:]
            else:
                elem = self.visit(child)
                if elem and members:
                    members += " "
                members += f"{elem}"
        if node.name:
            self.defs += f"<{node.name}> ::= {members};\n"
            return f"<{node.name}>"
        return f"{members}"

    def visit_Decl(self, node: c_ast.Decl):
        m = self.visit(node.children()[0][1])
        self.defs += f"<{node.name}> ::= {m};\n"
        return f"<{node.name}>"

    def visit_ArrayDecl(self, node: c_ast.ArrayDecl) -> str:
        member = self.visit(node.children()[0][1])
        index = self.visit(node.children()[1][1])
        return f"{member}{{{index}}}"

    def visit_Constant(self, node: c_ast.Constant) -> str:
        return f"{node.value}"

    def visit_IdentifierType(self, node: c_ast.IdentifierType) -> str:
        name = "_".join(node.names)
        return f"<{name}>"

    def visit_While(self, node: c_ast.While) -> str:
        cond = self.visit(node.children()[0][1])
        body = self.visit(node.children()[1][1])
        return f"({body})*"

    def visit_If(self, node: c_ast.If) -> str:
        cond = node.children()[0][1]

        # Convert lookaheads into expected bytes
        # as in `if (ReadUShort(FTell()) == 0x0121) ...`
        if isinstance(cond, c_ast.BinaryOp):
            binary_op: c_ast.BinaryOp = cond
            if binary_op.op == '==':
                lhs = binary_op.children()[0][1]
                rhs = binary_op.children()[1][1]
                if isinstance(rhs, c_ast.Constant):
                    constant = eval(rhs.value)
                else:
                    constant = None
                if constant and isinstance(lhs, c_ast.FuncCall):
                    funccall: c_ast.FuncCall = lhs
                    func = funccall.children()[0][1]
                    if (isinstance(func, c_ast.ID) and
                        func.name == 'ReadUShort'):
                        self.forced_bytes = [
                            # assume little endian
                            chr(constant % 256),
                            chr(constant // 256)
                        ]
                    if (isinstance(func, c_ast.ID) and
                        func.name == 'ReadUByte'):
                        self.forced_bytes = [
                            chr(constant),
                        ]

        iftrue = self.visit(node.children()[1][1])

        if len(node.children()) <= 2:
            return f"{iftrue}?"
        else:
            iffalse = self.visit(node.children()[2][1])
            return f"{iftrue} | {iffalse}"

visitor = BT2FandangoVisitor()
print(visitor.visit(ast))

<UINT> ::= <unsigned_int>;
<byte> ::= <char>;
<CHAR> ::= <char>;
<BYTE> ::= <byte>;
<uchar> ::= <unsigned_char>;
<ubyte> ::= <uchar>;
<UCHAR> ::= <uchar>;
<UBYTE> ::= <ubyte>;
<int16> ::= <short>;
<SHORT> ::= <short>;
<INT16> ::= <short>;
<uint16> ::= <unsigned_int16>;
<ushort> ::= <unsigned_short>;
<wchar_t> ::= <ushort>;
<USHORT> ::= <ushort>;
<UINT16> ::= <uint16>;
<WORD> ::= <ushort>;
<int32> ::= <int>;
<INT> ::= <int>;
<INT32> ::= <int>;
<LONG> ::= <long>;
<uint> ::= <unsigned_int>;
<uint32> ::= <uint>;
<ulong> ::= <unsigned_long>;
<UINT> ::= <uint>;
<UINT32> ::= <uint>;
<ULONG> ::= <ulong>;
<DWORD> ::= <uint>;
<int64> ::= <long_long>;
<quad> ::= <int64>;
<QUAD> ::= <int64>;
<INT64> ::= <int64>;
<__int64> ::= <int64>;
<uint64> ::= <unsigned_int64>;
<uquad> ::= <uint64>;
<UQUAD> ::= <uint64>;
<UINT64> ::= <uint64>;
<QWORD> ::= <uint64>;
<__uint64> ::= <uint64>;
<FLOAT> ::= <float>;
<DOUBLE> ::= <double>;
<hfloat> ::= <float>;
<HFLOAT> ::= <hfloat>;

<second> ::= <WORD>;
<minute> ::